In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
import subprocess

c:\Users\Mayur\Documents\College\4th sem\Exploratory\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(r"C:\Users\Mayur\Documents\College\4th sem\Exploratory\Data\Sentinel1_MODIS_SM_Masked_Urban_YellowRiver_11km.csv")

def clean_data(data):
    data = data.dropna(subset=['LAI', 'SoilMoisture', 'VH', 'VV', 'date'])
    data = data.groupby('date').mean(numeric_only=True).reset_index()
    data = data.drop(columns=['SoilRoughness_placeholder', 'Frequency_GHz'])
    data['Year'] = pd.to_datetime(data['date']).dt.year
    data['Month'] = pd.to_datetime(data['date']).dt.month
    data['Day'] = pd.to_datetime(data['date']).dt.day
    data = data.drop(columns=['date', 'Year'])
    data['Month'] = pd.to_numeric(data['Month'], errors='coerce')
    data['Day'] = pd.to_numeric(data['Day'], errors='coerce')
    data['Month_Sin'] = np.sin(2 * np.pi * (data['Month'] / 12))
    data['Month_Cos'] = np.cos(2 * np.pi * (data['Month'] / 12))
    data['Day_Sin'] = np.sin(2 * np.pi * (data['Day'] / 30))
    data['Day_Cos'] = np.cos(2 * np.pi * (data['Day'] / 30))
    data = data.drop(columns=['Month', 'Day'])

    data['VV'] = 10 ** (data['VV'] / 10)
    data['VH'] = 10 ** (data['VH'] / 10)

    scaler_vv_vh = StandardScaler()
    data[['VV', 'VH']] = scaler_vv_vh.fit_transform(data[['VV', 'VH']])

    scaler_sm_lai = StandardScaler()
    data[['SoilMoisture', 'LAI']] = scaler_sm_lai.fit_transform(data[['SoilMoisture', 'LAI']])

    data['IncidenceAngle'] = data['IncidenceAngle'] * 0.1 / data['SoilMoisture'].std()
    return data

data_clean = clean_data(data.copy())

In [3]:
feature_cols = ['SoilMoisture', 'LAI', 'IncidenceAngle', 'Month_Sin', 'Month_Cos', 'Day_Sin', 'Day_Cos']
target_cols_vv = ['VV']
target_cols_vh = ['VH']

X = data_clean[feature_cols].values
y_vv = data_clean[target_cols_vv].values
y_vh = data_clean[target_cols_vh].values

X_train, X_test, y_vv_train, y_vv_test, y_vh_train, y_vh_test = train_test_split(
    X, y_vv, y_vh, test_size=0.2, random_state=42
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_vv_train_tensor = torch.tensor(y_vv_train, dtype=torch.float32)
y_vv_test_tensor = torch.tensor(y_vv_test, dtype=torch.float32)
y_vh_train_tensor = torch.tensor(y_vh_train, dtype=torch.float32)
y_vh_test_tensor = torch.tensor(y_vh_test, dtype=torch.float32)

In [4]:
class WCMInspiredModule(nn.Module):
    def __init__(self):
        super(WCMInspiredModule, self).__init__()
        self.soil_transform = nn.Sequential(nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 1))
        self.veg_transform = nn.Sequential(nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 1))
        self.angle_transform = nn.Sequential(nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 1))

    def forward(self, SM, LAI, IncAngle):
        soil_sig = self.soil_transform(SM)
        veg_attn = torch.exp(-F.relu(self.veg_transform(LAI)))
        angle_mod = self.angle_transform(IncAngle)
        sigma0 = soil_sig * veg_attn + angle_mod
        return sigma0

In [5]:

class FullHybridModel(nn.Module):
    def __init__(self):
        super(FullHybridModel, self).__init__()
        self.phys_layer = WCMInspiredModule()
        self.temporal_mlp = nn.Sequential(
            nn.Linear(4, 16),
            nn.ELU(),
            nn.Linear(16, 8)
        )
        self.mlp = nn.Sequential(
            nn.Linear(1 + 3 + 8, 64),
            nn.ELU(),
            nn.Linear(64, 32),
            nn.ELU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        SM = x[:, 0:1]
        LAI = x[:, 1:2]
        IncAngle = x[:, 2:3]
        temporal = x[:, 3:]
        wcm_out = self.phys_layer(SM, LAI, IncAngle)
        temporal_out = self.temporal_mlp(temporal)
        x_cat = torch.cat([wcm_out, SM, LAI, IncAngle, temporal_out], dim=1)
        return self.mlp(x_cat)


In [6]:
def train_model(model, X_train, y_train, X_test, y_test, label):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                val_output = model(X_test)
                val_r2 = r2_score(y_test.numpy(), val_output.numpy())
            print(f"[{label}] Epoch {epoch}: Train Loss = {loss.item():.4f}, Val R² = {val_r2:.4f}")

    model.eval()
    with torch.no_grad():
        y_pred = model(X_test).numpy()
        r2 = r2_score(y_test.numpy(), y_pred)
    print(f"[{label}] Final R²: {r2:.4f}")
    return model

In [7]:
model_vv = FullHybridModel()
model_vv = train_model(model_vv, X_train_tensor, y_vv_train_tensor, X_test_tensor, y_vv_test_tensor, "VV")

[VV] Epoch 0: Train Loss = 1.1801, Val R² = -0.0556
[VV] Epoch 10: Train Loss = 0.9521, Val R² = 0.1073
[VV] Epoch 20: Train Loss = 0.8007, Val R² = 0.2098
[VV] Epoch 30: Train Loss = 0.6859, Val R² = 0.3371
[VV] Epoch 40: Train Loss = 0.6181, Val R² = 0.3911
[VV] Epoch 50: Train Loss = 0.5734, Val R² = 0.4357
[VV] Epoch 60: Train Loss = 0.5199, Val R² = 0.5100
[VV] Epoch 70: Train Loss = 0.4636, Val R² = 0.5887
[VV] Epoch 80: Train Loss = 0.4049, Val R² = 0.6671
[VV] Epoch 90: Train Loss = 0.3483, Val R² = 0.7372
[VV] Final R²: 0.7874


In [8]:
model_vh = FullHybridModel()
model_vh = train_model(model_vh, X_train_tensor, y_vh_train_tensor, X_test_tensor, y_vh_test_tensor, "VH")

[VH] Epoch 0: Train Loss = 0.9914, Val R² = 0.0681
[VH] Epoch 10: Train Loss = 0.8005, Val R² = 0.2787
[VH] Epoch 20: Train Loss = 0.6579, Val R² = 0.4155
[VH] Epoch 30: Train Loss = 0.5497, Val R² = 0.4990
[VH] Epoch 40: Train Loss = 0.4754, Val R² = 0.5496
[VH] Epoch 50: Train Loss = 0.4323, Val R² = 0.5782
[VH] Epoch 60: Train Loss = 0.3970, Val R² = 0.6189
[VH] Epoch 70: Train Loss = 0.3687, Val R² = 0.6502
[VH] Epoch 80: Train Loss = 0.3531, Val R² = 0.6472
[VH] Epoch 90: Train Loss = 0.3468, Val R² = 0.6383
[VH] Final R²: 0.6356


In [9]:


def plot_losses(train_losses, val_losses, label):
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(train_losses)), train_losses, label=f'{label} Train Loss')
    plt.plot(range(len(val_losses)), val_losses, label=f'{label} Val Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'{label} Training vs Validation Loss')
    plt.legend()
    plt.show()

train_losses_vv = []
val_losses_vv = []
train_losses_vh = []
val_losses_vh = []

def train_model_with_losses(model, X_train, y_train, X_test, y_test, label):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        train_losses_vv.append(loss.item())
        val_losses_vv.append(loss.item())  # Dummy for now, will add valid loss tracking

        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                val_output = model(X_test)
                val_r2 = r2_score(y_test.numpy(), val_output.numpy())
            print(f"[{label}] Epoch {epoch}: Train Loss = {loss.item():.4f}, Val R² = {val_r2:.4f}")
    plot_losses(train_losses_vv, label)

    model.eval()
    with torch.no_grad():
        y_pred = model(X_test).numpy()
        r2 = r2_score(y_test.numpy(), y_pred)
    print(f"[{label}] Final R²: {r2:.4f}")
    return model

In [10]:
def cross_validate_model(model_class, X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    r2_scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Convert numpy arrays to torch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

        model = model_class()
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, label="Cross-Validation")

        with torch.no_grad():
            y_pred = model(X_val_tensor).numpy()
            r2 = r2_score(y_val_tensor.numpy(), y_pred)
            r2_scores.append(r2)

    print(f"Average R² from {n_splits}-fold cross-validation: {np.mean(r2_scores):.4f}")

In [11]:
cross_validate_model(FullHybridModel, X_train, y_vv_train, n_splits=5)

[Cross-Validation] Epoch 0: Train Loss = 1.1307, Val R² = 0.0151
[Cross-Validation] Epoch 10: Train Loss = 0.9477, Val R² = 0.1280
[Cross-Validation] Epoch 20: Train Loss = 0.8012, Val R² = 0.2100
[Cross-Validation] Epoch 30: Train Loss = 0.6806, Val R² = 0.2716
[Cross-Validation] Epoch 40: Train Loss = 0.5880, Val R² = 0.3446
[Cross-Validation] Epoch 50: Train Loss = 0.5132, Val R² = 0.4392
[Cross-Validation] Epoch 60: Train Loss = 0.4491, Val R² = 0.5214
[Cross-Validation] Epoch 70: Train Loss = 0.3881, Val R² = 0.5843
[Cross-Validation] Epoch 80: Train Loss = 0.3369, Val R² = 0.6421
[Cross-Validation] Epoch 90: Train Loss = 0.3009, Val R² = 0.6778
[Cross-Validation] Final R²: 0.6922
[Cross-Validation] Epoch 0: Train Loss = 0.9519, Val R² = 0.0364
[Cross-Validation] Epoch 10: Train Loss = 0.7675, Val R² = 0.1739
[Cross-Validation] Epoch 20: Train Loss = 0.6446, Val R² = 0.2558
[Cross-Validation] Epoch 30: Train Loss = 0.5623, Val R² = 0.2909
[Cross-Validation] Epoch 40: Train Loss = 

In [12]:
cross_validate_model(FullHybridModel, X_train, y_vh_train, n_splits=5)

[Cross-Validation] Epoch 0: Train Loss = 1.1176, Val R² = 0.0373
[Cross-Validation] Epoch 10: Train Loss = 0.8737, Val R² = 0.2506
[Cross-Validation] Epoch 20: Train Loss = 0.6975, Val R² = 0.3877
[Cross-Validation] Epoch 30: Train Loss = 0.5954, Val R² = 0.4411
[Cross-Validation] Epoch 40: Train Loss = 0.5315, Val R² = 0.4682
[Cross-Validation] Epoch 50: Train Loss = 0.4911, Val R² = 0.4990
[Cross-Validation] Epoch 60: Train Loss = 0.4575, Val R² = 0.5461
[Cross-Validation] Epoch 70: Train Loss = 0.4263, Val R² = 0.5853
[Cross-Validation] Epoch 80: Train Loss = 0.4006, Val R² = 0.6090
[Cross-Validation] Epoch 90: Train Loss = 0.3820, Val R² = 0.6294
[Cross-Validation] Final R²: 0.6418
[Cross-Validation] Epoch 0: Train Loss = 1.0586, Val R² = -0.0029
[Cross-Validation] Epoch 10: Train Loss = 0.7672, Val R² = 0.2551
[Cross-Validation] Epoch 20: Train Loss = 0.5985, Val R² = 0.3945
[Cross-Validation] Epoch 30: Train Loss = 0.5120, Val R² = 0.4368
[Cross-Validation] Epoch 40: Train Loss =

In [13]:
def objective_vv(trial):
    # Define the hyperparameters to tune
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    hidden_units = trial.suggest_int('hidden_units', 16, 128, step=16)

    # Define the model with trial parameters
    class TunedHybridModel(nn.Module):
        def __init__(self):
            super(TunedHybridModel, self).__init__()
            self.phys_layer = WCMInspiredModule()
            
        self.temporal_mlp = nn.Sequential(
            nn.Linear(4, 16),
            nn.ELU(),
            nn.Linear(16, 8)
        )
        self.mlp = nn.Sequential(
                nn.Linear(8, hidden_units),
                nn.ELU(),
                nn.Linear(hidden_units, hidden_units // 2),
                nn.ELU(),
                nn.Linear(hidden_units // 2, 1)
            )

        def forward(self, x):
            SM = x[:, 0:1]
            LAI = x[:, 1:2]
            IncAngle = x[:, 2:3]
            temporal = x[:, 3:]
            wcm_out = self.phys_layer(SM, LAI, IncAngle)
            
        SM = x[:, 0:1]
        LAI = x[:, 1:2]
        IncAngle = x[:, 2:3]
        temporal = x[:, 3:]
        temporal_out = self.temporal_mlp(temporal)
        x_cat = torch.cat([wcm_out, SM, LAI, IncAngle, temporal_out], dim=1)
        
            return self.mlp(x_cat)

    # Perform 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    r2_scores = []

    for train_index, val_index in kf.split(X):
        # Split the data
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y_vv[train_index], y_vv[val_index]

        # Convert numpy arrays to torch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

        # Initialize and train the model
        model = TunedHybridModel()
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        for epoch in range(50):  # Reduced epochs for faster optimization
            model.train()
            optimizer.zero_grad()
            output = model(X_train_tensor)
            loss = criterion(output, y_train_tensor)
            loss.backward()
            optimizer.step()

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            val_output = model(X_val_tensor)
            r2 = r2_score(y_val_tensor.numpy(), val_output.numpy())
            r2_scores.append(r2)

    # Return the average R² score across folds
    return np.mean(r2_scores)

parameter_database = "sqlite:///C:/Users/Mayur/Documents/College/4th sem/Exploratory/Parameter_Databases/db.sqlite3"
# Run the optimization
study = optuna.create_study(direction='maximize', 
                            storage=parameter_database,
                            study_name="Hybrid Model Full Scale VV")
subprocess.Popen(["optuna-dashboard", parameter_database])
study.optimize(objective_vv, n_trials=400)

# Best hyperparameters
print("Best hyperparameters:", study.best_params)

# Train the final model with the best hyperparameters
best_params_vv = study.best_params
final_model_vv = FullHybridModel()  # Replace with the tuned model. Placed dummy here
final_model_vv = train_model(final_model_vv, X_train_tensor, y_vv_train_tensor, X_test_tensor, y_vv_test_tensor, "VV")

[I 2025-05-02 23:54:55,655] A new study created in RDB with name: Hybrid Model Full Scale VV
C:\Users\Mayur\AppData\Local\Temp\ipykernel_29036\1825325458.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
[I 2025-05-02 23:54:56,690] Trial 0 finished with value: 0.7106114983558655 and parameters: {'lr': 0.003048597418724064, 'hidden_units': 96}. Best is trial 0 with value: 0.7106114983558655.
C:\Users\Mayur\AppData\Local\Temp\ipykernel_29036\1825325458.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
[I 2025-05-02 23:54:57,392] Trial 1 finished with value: 0.00625950098037

Best hyperparameters: {'lr': 0.00536826389246822, 'hidden_units': 64}
[VV] Epoch 0: Train Loss = 1.0437, Val R² = -0.0288
[VV] Epoch 10: Train Loss = 0.8842, Val R² = 0.1159
[VV] Epoch 20: Train Loss = 0.7683, Val R² = 0.2550
[VV] Epoch 30: Train Loss = 0.6776, Val R² = 0.3515
[VV] Epoch 40: Train Loss = 0.5978, Val R² = 0.4524
[VV] Epoch 50: Train Loss = 0.5260, Val R² = 0.5421
[VV] Epoch 60: Train Loss = 0.4552, Val R² = 0.6344
[VV] Epoch 70: Train Loss = 0.3857, Val R² = 0.7157
[VV] Epoch 80: Train Loss = 0.3268, Val R² = 0.7756
[VV] Epoch 90: Train Loss = 0.2902, Val R² = 0.8000
[VV] Final R²: 0.7988


In [14]:
def objective_vh(trial):
    # Define the hyperparameters to tune
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    hidden_units = trial.suggest_int('hidden_units', 16, 128, step=16)

    # Define the model with trial parameters
    class TunedHybridModel(nn.Module):
        def __init__(self):
            super(TunedHybridModel, self).__init__()
            self.phys_layer = WCMInspiredModule()
            
        self.temporal_mlp = nn.Sequential(
            nn.Linear(4, 16),
            nn.ELU(),
            nn.Linear(16, 8)
        )
        self.mlp = nn.Sequential(
                nn.Linear(8, hidden_units),
                nn.ELU(),
                nn.Linear(hidden_units, hidden_units // 2),
                nn.ELU(),
                nn.Linear(hidden_units // 2, 1)
            )

        def forward(self, x):
            SM = x[:, 0:1]
            LAI = x[:, 1:2]
            IncAngle = x[:, 2:3]
            temporal = x[:, 3:]
            wcm_out = self.phys_layer(SM, LAI, IncAngle)
            
        SM = x[:, 0:1]
        LAI = x[:, 1:2]
        IncAngle = x[:, 2:3]
        temporal = x[:, 3:]
        temporal_out = self.temporal_mlp(temporal)
        x_cat = torch.cat([wcm_out, SM, LAI, IncAngle, temporal_out], dim=1)
        
            return self.mlp(x_cat)

    # Perform 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    r2_scores = []

    for train_index, val_index in kf.split(X):
        # Split the data
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y_vh[train_index], y_vh[val_index]

        # Convert numpy arrays to torch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

        # Initialize and train the model
        model = TunedHybridModel()
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        for epoch in range(50):  # Reduced epochs for faster optimization
            model.train()
            optimizer.zero_grad()
            output = model(X_train_tensor)
            loss = criterion(output, y_train_tensor)
            loss.backward()
            optimizer.step()

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            val_output = model(X_val_tensor)
            r2 = r2_score(y_val_tensor.numpy(), val_output.numpy())
            r2_scores.append(r2)

    # Return the average R² score across folds
    return np.mean(r2_scores)


# Run the optimization
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///C:/Users/Mayur/Documents/College/4th sem/Exploratory/Parameter_Databases/db.sqlite3",
                            study_name="Hybrid Model Full Scale VH")
study.optimize(objective_vv, n_trials=400)

# Best hyperparameters
print("Best hyperparameters:", study.best_params)

# Train the final model with the best hyperparameters
best_params_vh = study.best_params
final_model_vh = FullHybridModel()  # Replace with the tuned model. Placed dummy here
final_model_vh = train_model(final_model_vh, X_train_tensor, y_vv_train_tensor, X_test_tensor, y_vv_test_tensor, "VV")

[I 2025-05-03 00:02:55,946] A new study created in RDB with name: Hybrid Model Full Scale VH
C:\Users\Mayur\AppData\Local\Temp\ipykernel_29036\1825325458.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
[I 2025-05-03 00:02:56,535] Trial 0 finished with value: 0.03990292549133301 and parameters: {'lr': 0.00015401274798117431, 'hidden_units': 48}. Best is trial 0 with value: 0.03990292549133301.
C:\Users\Mayur\AppData\Local\Temp\ipykernel_29036\1825325458.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
[I 2025-05-03 00:02:57,178] Trial 1 finished with value: 0.1022937536

Best hyperparameters: {'lr': 0.008135744811663794, 'hidden_units': 48}
[VV] Epoch 0: Train Loss = 1.1340, Val R² = -0.0973
[VV] Epoch 10: Train Loss = 0.9195, Val R² = 0.1008
[VV] Epoch 20: Train Loss = 0.7701, Val R² = 0.2669
[VV] Epoch 30: Train Loss = 0.6603, Val R² = 0.3652
[VV] Epoch 40: Train Loss = 0.5828, Val R² = 0.4482
[VV] Epoch 50: Train Loss = 0.5283, Val R² = 0.5120
[VV] Epoch 60: Train Loss = 0.4726, Val R² = 0.5827
[VV] Epoch 70: Train Loss = 0.4146, Val R² = 0.6593
[VV] Epoch 80: Train Loss = 0.3599, Val R² = 0.7266
[VV] Epoch 90: Train Loss = 0.3141, Val R² = 0.7758
[VV] Final R²: 0.7995


In [ ]:
# Training with KFold validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FullHybridModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses, val_losses = [], []
for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_tensor)):
    train_X, val_X = X_train_tensor[train_idx], X_train_tensor[val_idx]
    train_y, val_y = y_vv_train_tensor[train_idx], y_vv_train_tensor[val_idx]
    
    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        preds = model(train_X.to(device))
        loss = criterion(preds, train_y.to(device))
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = model(val_X.to(device))
            val_loss = criterion(val_preds, val_y.to(device))

        if epoch % 10 == 0:
            print(f"Fold {fold+1}, Epoch {epoch}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
        if epoch == 99:
            train_losses.append(loss.item())
            val_losses.append(val_loss.item())

In [ ]:
# Overfitting check (loss curve)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title("Train vs Validation Loss (last epoch of each fold)")
plt.xlabel("Fold")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Final Predictions Visualization
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor.to(device)).cpu().numpy()

plt.figure(figsize=(8, 5))
plt.scatter(y_vv_test, predictions, alpha=0.5)
plt.plot([min(y_vv_test), max(y_vv_test)], [min(y_vv_test), max(y_vv_test)], color='red')
plt.xlabel("True VV")
plt.ylabel("Predicted VV")
plt.title("True vs. Predicted VV")
plt.grid(True)
plt.show()

residuals = y_vv_test - predictions
plt.figure(figsize=(8, 5))
plt.hist(residuals, bins=30, edgecolor='k')
plt.title("Residuals Distribution")
plt.xlabel("Residual (True - Predicted)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()